In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
from scipy.optimize import milp, LinearConstraint
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
def open_input(day='12', suffix="input"):
    with open(f"{day}-{suffix}", "r") as file:
        lines = file.readlines()
    data_raw = [line.replace("\n", "") for line in lines]
    data_raw = "\n".join(data_raw)
    return data_raw

input_raw = open_input()
input_raw

'0:\n#.#\n#.#\n###\n\n1:\n###\n.#.\n###\n\n2:\n###\n##.\n.##\n\n3:\n###\n.##\n..#\n\n4:\n###\n###\n#..\n\n5:\n.##\n##.\n#..\n\n44x48: 35 35 39 42 41 32\n50x45: 39 54 38 26 43 40\n50x50: 43 56 40 46 42 29\n46x36: 39 39 52 44 37 45\n49x46: 59 48 64 65 54 58\n41x40: 31 34 16 26 29 32\n43x35: 30 21 24 25 32 22\n50x48: 37 49 44 47 45 33\n47x41: 27 38 22 40 35 32\n49x49: 47 40 49 37 47 36\n46x48: 39 36 37 40 41 46\n39x41: 45 38 36 44 45 38\n40x48: 45 36 43 46 59 73\n48x45: 40 38 50 41 38 33\n42x50: 59 50 66 44 56 44\n46x49: 38 26 46 52 40 38\n47x49: 43 36 32 44 43 42\n47x39: 53 45 46 49 36 56\n50x46: 61 69 52 63 57 50\n35x37: 18 17 24 20 31 21\n48x46: 62 51 58 58 58 52\n41x40: 48 52 38 32 35 48\n47x50: 51 61 63 66 65 55\n41x40: 43 39 38 42 50 40\n37x46: 46 44 33 42 50 48\n43x36: 33 37 32 53 49 35\n43x46: 50 51 42 56 51 57\n50x46: 36 45 34 42 37 46\n49x46: 59 60 67 52 59 46\n35x49: 26 26 24 40 35 25\n48x39: 48 47 52 45 48 48\n38x40: 24 24 33 28 24 22\n48x48: 38 36 39 47 43 52\n50x47: 62 57 52

In [4]:
input_test_raw = open_input( suffix="test")
input_test_raw

'0:\n###\n##.\n##.\n\n1:\n###\n##.\n.##\n\n2:\n.##\n###\n##.\n\n3:\n##.\n###\n##.\n\n4:\n###\n#..\n###\n\n5:\n###\n.#.\n###\n\n4x4: 0 0 0 0 2 0\n12x5: 1 0 1 0 2 2\n12x5: 1 0 1 0 3 2'

In [11]:
def preprocess_data (data):
    # dtype='U10'
    
    presents = []

    for present in data.split("\n\n")[:-1]:
        presents.append(np.array([list(row) for row in present.split("\n")[1:]]))

    regions = []

    for region in data.split("\n\n")[-1].split("\n"):
        dims = [int(i) for i in region.split(": ")[0].split("x") ]
        shapes = [int(i) for i in region.split(": ")[1].split(" ") ]
        regions.append((dims, shapes))

    return presents, regions
test_data = preprocess_data(input_test_raw)
input_data = preprocess_data(input_raw)
display(test_data)

([array([['#', '#', '#'],
         ['#', '#', '.'],
         ['#', '#', '.']], dtype='<U1'),
  array([['#', '#', '#'],
         ['#', '#', '.'],
         ['.', '#', '#']], dtype='<U1'),
  array([['.', '#', '#'],
         ['#', '#', '#'],
         ['#', '#', '.']], dtype='<U1'),
  array([['#', '#', '.'],
         ['#', '#', '#'],
         ['#', '#', '.']], dtype='<U1'),
  array([['#', '#', '#'],
         ['#', '.', '.'],
         ['#', '#', '#']], dtype='<U1'),
  array([['#', '#', '#'],
         ['.', '#', '.'],
         ['#', '#', '#']], dtype='<U1')],
 [([4, 4], [0, 0, 0, 0, 2, 0]),
  ([12, 5], [1, 0, 1, 0, 2, 2]),
  ([12, 5], [1, 0, 1, 0, 3, 2])])

In [18]:
def solution(data):
    solution = 0
    
    presents, regions = data
    sizes = {}
    for idx, present in enumerate(presents):
        unique, counts = np.unique(present, return_counts=True)
        res = {s:n.item() for s, n in zip(unique, counts)}
        sizes[idx] = res["#"]

    for shape, reqs in regions:
        tiles = shape[0]*shape[1]
        space_reqs = sum([p*sizes[idx] for idx, p in enumerate(reqs) ])
        if tiles < space_reqs:
            print(shape, reqs)
        else:
            solution += 1

    return solution

sol_test = solution(test_data)
print("Test Solution:", sol_test)
sol_input = solution(input_data)
print("Input Solution:", sol_input)

Test Solution: 3
[46, 36] [39, 39, 52, 44, 37, 45]
[49, 46] [59, 48, 64, 65, 54, 58]
[39, 41] [45, 38, 36, 44, 45, 38]
[40, 48] [45, 36, 43, 46, 59, 73]
[42, 50] [59, 50, 66, 44, 56, 44]
[47, 39] [53, 45, 46, 49, 36, 56]
[50, 46] [61, 69, 52, 63, 57, 50]
[48, 46] [62, 51, 58, 58, 58, 52]
[41, 40] [48, 52, 38, 32, 35, 48]
[47, 50] [51, 61, 63, 66, 65, 55]
[41, 40] [43, 39, 38, 42, 50, 40]
[37, 46] [46, 44, 33, 42, 50, 48]
[43, 36] [33, 37, 32, 53, 49, 35]
[43, 46] [50, 51, 42, 56, 51, 57]
[49, 46] [59, 60, 67, 52, 59, 46]
[48, 39] [48, 47, 52, 45, 48, 48]
[50, 47] [62, 57, 52, 69, 58, 67]
[40, 50] [49, 58, 52, 58, 41, 51]
[49, 41] [49, 44, 56, 56, 59, 44]
[43, 45] [51, 58, 46, 46, 43, 55]
[45, 42] [50, 48, 59, 44, 40, 50]
[46, 43] [53, 64, 50, 48, 42, 46]
[48, 50] [52, 60, 49, 66, 72, 75]
[35, 36] [36, 37, 31, 31, 31, 26]
[44, 49] [43, 53, 74, 59, 44, 61]
[42, 43] [38, 39, 54, 52, 50, 46]
[41, 45] [52, 50, 47, 46, 46, 41]
[48, 50] [62, 56, 63, 62, 66, 60]
[48, 41] [46, 48, 58, 51, 52, 4

In [14]:



def solution2(data):
    solution = 0
    
    memory = {}

    def count_all_paths(start, fft, dac):
        if start == 'out':
            return 1 if fft and dac else 0
        
        if (start, fft, dac) in memory.keys():
            return memory[(start, fft, dac)]

        if start == 'fft':
            res = sum([count_all_paths(next, True, dac) for next in data[start]])
            memory[(start, fft, dac)] = res
            return res
        elif start == 'dac':
            res = sum([count_all_paths(next, fft, True) for next in data[start]])
            memory[(start, fft, dac)] = res
            return res
        else:
            res = sum([count_all_paths(next, fft, dac) for next in data[start]])
            memory[(start, fft, dac)] = res
            return res

    solution = count_all_paths("svr", False, False)
    

    return solution


sol_test = solution2(test_data2)
print("Test Solution:", sol_test)
sol_input = solution2(input_data)
print("Input Solution:", sol_input)

Test Solution: 2
Input Solution: 290219757077250
